In [1]:
import chromadb
import pandas as pd

# Connect to your ChromaDB host
client = chromadb.HttpClient(host="170.64.231.135", port=8000)

client.list_collections()

[Collection(name=ghg_collection)]

In [3]:

collection = client.get_collection(name="ghg_collection")

# Fetch all documents (adjust limit if needed)
results = collection.get(include=["documents", "metadatas", "embeddings"], limit=10000)

# Convert to DataFrame
df = pd.DataFrame({
    "document": results["documents"],
    "metadata": results["metadatas"],
    "embedding": results["embeddings"]
})

# Save to CSV
df.to_csv("ghg_chunks_export.csv", index=False)

print("✅ Exported to ghg_chunks_export.csv")

✅ Exported to ghg_chunks_export.csv


In [4]:
import pandas as pd
chunk_text = pd.read_csv("ghg_chunks_export.csv")


chunk_text.head()

,document,metadata,embedding


In [5]:
from groq import Client
import os

# Set API key
os.environ["GROQ_API_KEY"] = "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"
groq_client = Client(api_key=os.getenv("GROQ_API_KEY"))

# Build context from all chunks (filtered to stay under 4000 tokens)
chunk_column = "document" if "document" in chunk_text.columns else chunk_text.columns[0]
chunks = chunk_text[chunk_column].dropna().astype(str).tolist()

# Keep appending chunks until reaching character limit (~16,000 chars = ~4k tokens)
max_chars = 16000
selected_chunks = []
current_len = 0

for c in chunks:
    if current_len + len(c) > max_chars:
        break
    selected_chunks.append(c)
    current_len += len(c)

filtered_context = "\n---\n".join(selected_chunks)

# Prompt
prompt = f"""
You are a GHG emissions consultant and expert in international and Australian climate reporting frameworks.

Using the full context below, generate a list of **question and answer (Q&A) pairs** that summarize key insights, regulatory obligations, and challenges for companies.

Context:
{filtered_context}

Format the output like this:
Q1: ...
A1: ...
Q2: ...
A2: ...
(...continue...)

Only include the question–answer pairs in your response.

Begin:
"""

# Query Groq
chat_completion = groq_client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a knowledgeable climate consultant generating clear and accurate Q&A pairs."},
        {"role": "user", "content": prompt}
    ]
)

# Show result
response = chat_completion.choices[0].message.content
print("🧠 Generated Q&A Pairs:\n")
print(response)

🧠 Generated Q&A Pairs:

Q1: What is the primary goal of the Paris Agreement, and how does it relate to companies' GHG emissions reporting?
A1: The Paris Agreement aims to limit global warming to well below 2°C and pursue efforts to limit it to 1.5°C above pre-industrial levels. Companies play a crucial role in achieving this goal by reporting their GHG emissions transparently and taking actions to reduce them.

Q2: Which international climate reporting framework is widely adopted by companies, and what are its key components?
A2: The Greenhouse Gas Protocol (GHGP) is the most widely used international framework for GHG emissions reporting. Its key components include the Corporate Standard, which outlines requirements for scope 1, 2, and 3 emissions reporting, and the Product Standard, which provides guidelines for product life cycle assessment.

Q3: What is the significance of scope 1, 2, and 3 emissions in GHG reporting, and how do companies typically categorize their emissions?
A3: S